In [1]:
import numpy as np
import pandas as pd

import tensorflow_decision_forests as tfdf
import tensorflow as tf

2023-07-16 10:31:05.792286: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 10:31:07.111429: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 10:31:07.121561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 10:31:10.556011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_df = pd.read_csv('/home/karen/Documents/Datasets/Multi-Label_Classification_with_an_Enzyme_Substrate_Dataset/train.csv')
test_df = pd.read_csv('/home/karen/Documents/Datasets/Multi-Label_Classification_with_an_Enzyme_Substrate_Dataset/test.csv')
submission_df = pd.read_csv('/home/karen/Documents/Datasets/Multi-Label_Classification_with_an_Enzyme_Substrate_Dataset/sample_submission.csv')

In [3]:
train_df

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
0,0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,...,4.794537,35.527357,0,0,1,1,0,0,0,0
1,1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,...,13.825658,44.707310,0,0,0,1,1,0,0,0
2,2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,...,17.964475,45.660120,0,0,1,1,0,0,1,0
3,3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,...,31.961948,87.509997,0,0,1,1,0,0,0,0
4,4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,...,9.589074,33.333333,2,2,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14833,14833,632.207041,10.911303,6.579933,9.179964,4.653583,6.030052,3.670528,1.770579,32.971529,...,18.947452,61.376610,0,0,1,1,0,0,0,0
14834,14834,62.568425,2.642734,1.446898,1.446898,0.879497,0.879497,0.174620,0.000000,0.000000,...,0.000000,10.000000,0,0,0,1,0,1,0,0
14835,14835,981.327476,10.363081,6.146219,6.146219,4.700576,4.700576,3.064846,2.133897,17.248535,...,0.000000,66.666667,0,0,1,1,0,0,0,0
14836,14836,299.171248,9.949161,6.589761,7.848913,5.276568,5.476436,3.978973,2.299833,45.623794,...,9.088795,45.583333,0,0,0,1,1,0,0,0


In [4]:
train_df.describe()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,...,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,7418.500000,515.153604,9.135189,5.854307,6.738497,4.432570,5.253221,3.418749,1.773472,29.202823,...,13.636941,49.309959,0.458215,0.459226,0.667745,0.798962,0.313789,0.279081,0.144831,0.151570
std,4283.505982,542.456370,6.819989,4.647064,5.866444,3.760516,4.925065,3.436208,1.865898,31.728679,...,14.598554,29.174824,0.667948,0.668111,0.471038,0.400790,0.464047,0.448562,0.351942,0.358616
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.430556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3709.250000,149.103601,4.680739,2.844556,2.932842,1.949719,2.034468,1.160763,0.503897,5.969305,...,4.794537,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7418.500000,290.987941,6.485270,4.052701,4.392859,2.970427,3.242775,1.948613,1.073261,17.353601,...,9.589074,41.666667,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,11127.750000,652.652585,11.170477,7.486791,8.527859,5.788793,6.609350,4.502070,2.534281,44.876559,...,14.912664,56.090650,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,14837.000000,4069.959780,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.072810,363.705954,...,115.406157,384.450519,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
primary_labels = ["EC1", "EC2"]
secondary_labels = ["EC3", "EC4", "EC5", "EC6"]
non_feature_columns = ["id"]

In [6]:
def to_tf_dataset(pd_dataset: pd.DataFrame, label_keys: list[str], droped_features: list[str]) -> tf.data.Dataset:
    features = dict(pd_dataset.drop(label_keys + droped_features, axis=1))
    labels = dict(pd_dataset[label_keys])
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(100)

train_tf = to_tf_dataset(train_df, label_keys=primary_labels, droped_features=non_feature_columns + secondary_labels)
test_tf = to_tf_dataset(test_df, label_keys=[], droped_features=non_feature_columns)

In [7]:
print(train_tf)

<_BatchDataset element_spec=({'BertzCT': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi1n': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi1v': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi2n': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi2v': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi3v': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Chi4n': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'EState_VSA1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'EState_VSA2': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ExactMolWt': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FpDensityMorgan1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FpDensityMorgan2': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FpDensityMorgan3': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'HallKier

In [8]:
model = tfdf.keras.GradientBoostedTreesModel(
    multitask=[tfdf.keras.MultiTaskItem(label=l, task=tfdf.keras.Task.CLASSIFICATION) for l in primary_labels],
    verbose=1,
)
model.fit(train_tf)

Use /tmp/tmpy_f2pjhd as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.651373. Found 14838 examples.
Training model...
Model trained in 0:00:03.714820
Compiling model...


[INFO 23-07-16 10:31:25.3087 EAT kernel.cc:1243] Loading model from path /tmp/tmpy_f2pjhd/model/ with prefix 7cf6db54522d486e_0
[INFO 23-07-16 10:31:25.3277 EAT abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-07-16 10:31:25.3278 EAT kernel.cc:1075] Use fast generic engine
[INFO 23-07-16 10:31:25.3330 EAT kernel.cc:1243] Loading model from path /tmp/tmpy_f2pjhd/model/ with prefix 7cf6db54522d486e_1
[INFO 23-07-16 10:31:25.3348 EAT kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [9]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "MULTITASKER"
Task: CLASSIFICATION
Label: "EC1"

Input Features (31):
	BertzCT
	Chi1
	Chi1n
	Chi1v
	Chi2n
	Chi2v
	Chi3v
	Chi4n
	EState_VSA1
	EState_VSA2
	ExactMolWt
	FpDensityMorgan1
	FpDensityMorgan2
	FpDensityMorgan3
	HallKierAlpha
	HeavyAtomMolWt
	Kappa3
	MaxAbsEStateIndex
	MinEStateIndex
	NumHeteroatoms
	PEOE_VSA10
	PEOE_VSA14
	PEOE_VSA6
	PEOE_VSA7
	PEOE_VSA8
	SMR_VSA10
	SMR_VSA5
	SlogP_VSA3
	VSA_EState9
	fr_COO
	fr_COO2

No weights

Variable Importance disabled i.e. compute_oob_variable_importances=false.
Cannot compute model self evaluation:This model does not support evaluation reports.
model #0
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION


In [10]:
prediction = model.predict(test_tf)

prediction

99/99 [==============================] - 0s 3ms/step


{'EC1': array([[0.47080842],
        [0.80730313],
        [0.77015865],
        ...,
        [0.44579405],
        [0.5141975 ],
        [0.4168353 ]], dtype=float32),
 'EC2': array([[0.7818088 ],
        [0.85070294],
        [0.7597029 ],
        ...,
        [0.8409997 ],
        [0.84569997],
        [0.8245511 ]], dtype=float32)}

In [11]:
prediction_pd = pd.DataFrame({
    "id": test_df["id"],
    "EC1": prediction["EC1"].flatten(),
    "EC2": prediction["EC2"].flatten(),
})

prediction_pd.to_csv("submission.csv",index=False)

prediction_pd

,id,EC1,EC2
0,14838,0.470808,0.781809
1,14839,0.807303,0.850703
2,14840,0.770159,0.759703
3,14841,0.706230,0.824318
4,14842,0.794809,0.750287
...,...,...,...
9888,24726,0.631759,0.759918
9889,24727,0.775003,0.813876
9890,24728,0.445794,0.841000
9891,24729,0.514198,0.845700
